In [1]:
import os
import mlflow
import torch
import flash

from flash.text import TextClassificationData,TextClassifier

/opt/conda/envs/test1/lib/python3.8/site-packages/pytorch_tabular/models/mixture_density/mdn.py:25: UserWarning: Wandb not installed. WandbLogger will not work.
  warnings.warn("Wandb not installed. WandbLogger will not work.")


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
mlflow.environment_variables.MLFLOW_S3_ENDPOINT_URL = "http://localhost:9000"

In [3]:
data_module = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="../data/imdb/train.csv",
    test_file="../data/imdb/test.csv",
    val_file="../data/imdb/valid.csv",
    batch_size=64
)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/opt/conda/envs/test1/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
model = TextClassifier(backbone="prajjwal1/bert-tiny",
                       num_classes=data_module.num_classes,
                       labels=data_module.labels)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [5]:
trainer = flash.Trainer(max_epochs=3,
                        gpus=torch.cuda.device_count())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
EXPERIMENT_NAME = "full_fledged"
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print(exp.experiment_id)

737308629393873003


In [7]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name="with_auto_log") as model_tracking_run:
    trainer.finetune(model,data_module,strategy="freeze")

2023/03/23 11:38:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
2023/03/23 11:39:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/tmp/tmp77u1f9xz/model/data, flavor: pytorch), fall back to return ['torch==2.0.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


In [8]:
run_id = model_tracking_run.info.run_id
run_id , mlflow.get_run(run_id).info.lifecycle_stage

('144069acdaeb4cee8798e520ec72d713', 'active')

In [9]:
logged_model_uri = f"runs:/{run_id}/model"
logged_model_uri

'runs:/144069acdaeb4cee8798e520ec72d713/model'

In [10]:
sample_data = ["Worst movie ever seen.",
               "No time lag"]
sample_module = TextClassificationData.from_lists(predict_data=sample_data,
                                                  batch_size=1,
                                                  )

In [11]:
loaded_model = mlflow.pytorch.load_model(logged_model_uri)
trainer.predict(model=loaded_model,
                datamodule=sample_module,
                output="labels")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 351it [00:00, ?it/s]

[['negative'], ['positive']]

In [12]:
model_registry = mlflow.register_model(logged_model_uri,"nlp_model")
model_registry.name,model_registry.version

Successfully registered model 'nlp_model'.
2023/03/23 11:45:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nlp_model, version 1
Created version '1' of model 'nlp_model'.


('nlp_model', '1')